# Sequence to Sequence (a.k.a. seq2seq)

**학습목표**
* Encoder Decoder 구조를 이해하고 구현할 줄 안다.
* Seq2Seq에 필요한 전처리를 이해한다.
* **데이터 부족**과, **긴 문장**을 겪어본다.

![이런거](https://raw.githubusercontent.com/KerasKorea/KEKOxTutorial/master/media/28_1.png)
---------------------------------
edu.rayleigh@gmail.com
Special Thanks to : 숙번님 ( [봉수골 개발자 이선비](https://www.youtube.com/channel/UCOAyyrvi7tnCAz7RhH98QCQ) )

In [58]:
!wget http://www.manythings.org/anki/kor-eng.zip

--2021-02-06 18:45:29--  http://www.manythings.org/anki/kor-eng.zip
Resolving www.manythings.org (www.manythings.org)... 172.67.173.198, 104.21.55.222, 2606:4700:3036::ac43:adc6, ...
Connecting to www.manythings.org (www.manythings.org)|172.67.173.198|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 143703 (140K) [application/zip]
Saving to: ‘kor-eng.zip.2’

kor-eng.zip.2       100%[===================>] 140.33K  --.-KB/s    in 0.1s    

2021-02-06 18:45:29 (1.26 MB/s) - ‘kor-eng.zip.2’ saved [143703/143703]



In [59]:
import zipfile
kor_eng = zipfile.ZipFile('kor-eng.zip')
kor_eng.extractall()
kor_eng.close()

In [60]:
import pandas as pd
temp = pd.read_table('kor.txt', names=['Eng', 'Kor', 'license'])
temp.shape

(3648, 3)

In [61]:
temp.head()

,Eng,Kor,license
0,Go.,가.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
1,Hi.,안녕.,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
2,Run!,뛰어!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
3,Run.,뛰어.,CC-BY 2.0 (France) Attribution: tatoeba.org #4...
4,Who?,누구?,CC-BY 2.0 (France) Attribution: tatoeba.org #2...


In [62]:
eng_sent = temp['Eng'].tolist()
kor_sent = temp['Kor'].tolist()

In [63]:
print(eng_sent[1000])
print(kor_sent[1000])

I don't sleep a lot.
나는 잠이 별로 없어.


# 데이터 준비
0. 단어와 구두점 사이 공백 만들기
1. sos 와 eos
1. tokenizing, idx_seq, padding

## 0. 단어와 구두점 사이 공백 만들기


In [64]:
import unicodedata
import re
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')
def eng_preprocessor(sent):
    # 위에서 구현한 함수를 내부적으로 호출
    sent = unicode_to_ascii(sent.lower())

    # 단어와 구두점 사이에 공백을 만듭니다.
    # Ex) "he is a boy." => "he is a boy ."
    sent = re.sub(r"([?.!,'¿])", r" \1 ", sent)

    # (a-z, A-Z, ".", "?", "!", ",") 이들을 제외하고는 전부 공백으로 변환합니다.
    sent = re.sub(r"[^a-zA-Z!.?']+", r" ", sent)

    sent = re.sub(r"\s+", " ", sent)
    return sent

def kor_preprocessor(sent):
    # 위에서 구현한 함수를 내부적으로 호출
    sent = unicode_to_ascii(sent.lower())

    # 단어와 구두점 사이에 공백을 만듭니다.
    # Ex) "he is a boy." => "he is a boy ."
    sent = re.sub(r"([?.!,'¿])", r" \1 ", sent)

    sent = re.sub(r"\s+", " ", sent)
    return sent

In [65]:
eng_preprocessor("I'm just a poor boy.")

"i ' m just a poor boy . "

In [66]:
eng_sent = [ eng_preprocessor(sent) for sent in eng_sent ]
kor_sent = [ kor_preprocessor(sent) for sent in kor_sent ]

In [67]:
print(eng_sent[1000])
print(kor_sent[1000])

i don ' t sleep a lot . 
나는 잠이 별로 없어 . 


## 1. sos 와 eos
1. sos : start of speech
2. eos : end of speech

In [68]:
######################
### Your Code here ###
######################

## 영어 문장 전 후에 <sos>와 <eos>를 추가할 것
## 띄어쓰기 주의!

eng_sent = [f"<sos> {eng} <eos>" for eng in eng_sent]
eng_sent[1000]

"<sos> i don ' t sleep a lot .  <eos>"

## 2. Tokenizing, idx_seq, padding

In [69]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [70]:
######################
### Your Code here ###
######################

# Tokenizing    # 한국어는 lower = False
tokenizer_en = Tokenizer(filters="", lower=True)
tokenizer_en.fit_on_texts(eng_sent)
tokenizer_kr = Tokenizer(filters="", lower=False)
tokenizer_kr.fit_on_texts(kor_sent)

In [71]:
######################
### Your Code here ###
######################

# Index Sequence
eng_seq = tokenizer_en.texts_to_sequences(eng_sent)
kor_seq = tokenizer_kr.texts_to_sequences(kor_sent)

print(eng_seq[1000])
print(kor_seq[1000])

[1, 4, 22, 5, 13, 252, 11, 148, 3, 2]
[7, 714, 445, 19, 1]


In [72]:
######################
### Your Code here ###
######################
## 최대 문장 길이에 맞춰지도록 할 것.
# padding
eng_pad = pad_sequences(eng_seq) # 최대 문장 길이에 패딩에 맞춰지게 됨.
kor_pad = pad_sequences(kor_seq)

print(eng_pad.shape)
print(kor_pad.shape)

(3648, 104)
(3648, 95)


In [73]:
# tokenizer에서 0 index가 구성되어있지 않지만, 
# pad_sequence에서 pad의 의미로 0을 사용하고 있어서, 전체 사이즈를 구할 때, +1을 해준다.

eng_vocab_size = len(tokenizer_en.word_index) + 1
kor_vocab_size = len(tokenizer_kr.word_index) + 1
print("영어 단어 집합의 크기: {:d}\n한국어 단어 집합의 크기: {:d}".format(eng_vocab_size, kor_vocab_size))

영어 단어 집합의 크기: 2484
한국어 단어 집합의 크기: 5551


# 모델링!

1. 모든 임베딩 레이어는 128개 차원으로 구성.
2. 인코더도 디코더도 GRU, 히든스테이트 512로 구성.
3. 디코더의 GRU 뒤에는 Fully Conneceted layer 사용. 노드 512개
4. 적절한 아웃풋레이어
    * 매 시점, 가장 적절한 단어가 무엇일지 분류 한다고 생각하면 됨!

In [74]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, GRU

In [75]:
######################
### Your Code here ###
######################

# 혹시 이미 그려둔 그래프가 있다면 날려줘!
tf.keras.backend.clear_session()

# 한국어 단어 집합의 크기 : 5551, (50000, 95)
# 영어 단어 집합의 크기 : 2484, (50000, 104)
# 영어 문장은 길이가 104이지만,
# 디코더의 인풋으로 넣을때는 맨 뒤의 <eos>를 떼고 길이 103의 문장을
# 디코더의 아웃풋은 맨 앞의 <eos>를 떼고 길이 103의 문장으로 준비해야 함.

# Encoder
enc_X = tf.keras.layers.Input(shape=[kor_pad.shape[1]])
enc_E = tf.keras.layers.Embedding(kor_vocab_size, 128)(enc_X) # 토큰수, 차원수
enc_S_full, enc_S = tf.keras.layers.GRU(512, return_sequences=True, return_state=True)(enc_E)
## 이제는 enc_S_full을 쓴다!

# Decoder
dec_X = tf.keras.layers.Input(shape=[eng_pad.shape[1]-1])
dec_E = tf.keras.layers.Embedding(eng_vocab_size, 128)(dec_X) # 토큰수, 차원수
dec_H = tf.keras.layers.GRU(512, return_sequences=True)(dec_E, initial_state=enc_S)

#####################
## Attention layer ##
#####################
key = enc_S_full  # 인코더의 히든스테이트를 key로 활용한다. (95, 512)
value = enc_S_full  # 인코더의 히든스테이트를 value로 활용한다. (95, 512)
query = dec_H  # 디코더의 히든스테이트를 query로 활용한다. (103, 256)

# 1. 어텐션 스코어(Attention Score)를 구한다.
score = tf.matmul(query, key, transpose_b=True)
# 연산 결과: (103, 512) * (512, 95) => (103, 95)
# 영어 103개 step 각각에서  한국어 95개 step 전부의 스코어

# 2. 소프트맥스(softmax) 함수를 통해 어텐션 분포(Attention Distribution)를 구한다.
att_dist = tf.nn.softmax(score, axis=-1)
# 연산 결과 : (103, 95) 그대로
# 위의 영어 103개 step 각각에서의 attention score들이 softmax를 통과하여 합계 1이 되었음.

# 3. 각 인코더의 어텐션 가중치와 은닉 상태를 가중합하여 어텐션 값(Attention Value)을 구한다.
att_value = tf.matmul(att_dist, value)
# 연산 결과: (103, 95) * (95, 512) => (103,512)
# value의 row 들은  att_dist(softmax)값이 높은 것들이 더 가중되어 합산됨.

### 사실은, 아래 한줄의 코드로도 가능.
# att_value = tf.keras.layers.Attention()([query, key])


# 4. 어텐션 값과 디코더의 t 시점의 은닉 상태를 연결한다.(Concatenate)

dec_H = tf.keras.layers.Concatenate()([att_value, dec_H])
# 5. 출력층 연산의 입력이 되는 dec_H를 계산.
dec_H = tf.keras.layers.Dense(512, activation='tanh')(dec_H)

dec_Y = tf.keras.layers.Dense(kor_vocab_size, activation="softmax")(dec_H)
model = tf.keras.models.Model([enc_X, dec_X], dec_Y)
# 텍스트는 index이고(원핫인코딩을 안했고)
# 아웃풋레이어는 분류문제 처럼 노드가 준비되어 있다면
# sparse categorical crossentropy
model.compile(loss='sparse_categorical_crossentropy',
              optimizer = tf.keras.optimizers.RMSprop(),
              metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 95)]         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 103)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 95, 128)      710528      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 103, 128)     317952      input_2[0][0]                    
______________________________________________________________________________________________

In [76]:
######################
### Your Code here ###
######################
## 학습 시킬 것!

# decoder의 인풋은 마지막 <eos>를 뗀다.
# decoder의 아웃풋 학습시엔 처음의 <sos>를 뗀다.
model.fit([kor_pad, eng_pad[:, :-1]], eng_pad[:, 1:], shuffle=True, 
          batch_size=128, epochs=40)

Epoch 1/40
29/29 [==============================] - 8s 171ms/step - loss: 2.5990 - accuracy: 0.7911
Epoch 2/40
29/29 [==============================] - 5s 174ms/step - loss: 0.6553 - accuracy: 0.9141
Epoch 3/40
29/29 [==============================] - 5s 177ms/step - loss: 0.4750 - accuracy: 0.9220
Epoch 4/40
29/29 [==============================] - 5s 180ms/step - loss: 0.4464 - accuracy: 0.9260
Epoch 5/40
29/29 [==============================] - 5s 180ms/step - loss: 0.4096 - accuracy: 0.9324
Epoch 6/40
29/29 [==============================] - 5s 180ms/step - loss: 0.3979 - accuracy: 0.9338
Epoch 7/40
29/29 [==============================] - 5s 183ms/step - loss: 0.3720 - accuracy: 0.9364
Epoch 8/40
29/29 [==============================] - 5s 184ms/step - loss: 0.3691 - accuracy: 0.9357
Epoch 9/40
29/29 [==============================] - 5s 181ms/step - loss: 0.3492 - accuracy: 0.9384
Epoch 10/40
29/29 [==============================] - 5s 180ms/step - loss: 0.3429 - accuracy: 0.9386

In [77]:
import numpy as np

# 한국어 단어 집합의 크기 : 5551, (50000, 95)
# 영어 단어 집합의 크기 : 2484, (50000, 104)

def translate(kor):
    # eng => index => pad
    kor_seq = tokenizer_kr.texts_to_sequences([kor])
    kor_pad = tf.keras.preprocessing.sequence.pad_sequences(kor_seq, maxlen=95)

    eng = []
    for n in range(104-1):
        # kor => index => pad
        eng_seq = tokenizer_en.texts_to_sequences([['<sos>'] + eng])
        eng_pad = tf.keras.preprocessing.sequence.pad_sequences(eng_seq, maxlen=104-1)
        eng_next = model.predict([kor_pad, eng_pad])

        # onehot -> index -> word
        eng = [tokenizer_en.index_word[i] for i in np.argmax(eng_next[0], axis=1) if i != 0]
        # 번역된 word 선택
        eng = eng[:n+1]
        
        if eng[-1] == '<eos>':
            break

    return eng

In [79]:
import random

# 랜덤 10개
indices = list(range(3648))
random.shuffle(indices)

for n in indices[:10]:
    print(f"한국어: {kor_sent[n]}\n영어: {eng_sent[n]}")
    print(f"번역: {' '.join(translate(kor_sent[n])[:-1])}")
    print()

한국어: 난 내 집이 좋아 . 
영어: <sos> i love my home .  <eos>
번역: i ' m not sure i can go to the party .

한국어: 나는 약을 여섯 시간마다 먹어야 해 . 
영어: <sos> i have to take my medicine every six hours .  <eos>
번역: i ' m not sure i can go to the party .

한국어: 톰은 잊어버려 . 
영어: <sos> forget tom .  <eos>
번역: tom was diagnosed with asd .

한국어: 나는 뭔가가 잘못되었다고 생각했다 . 
영어: <sos> i thought something was wrong .  <eos>
번역: i ' m sorry .

한국어: 법 이전에 모두는 평등해 . 
영어: <sos> everybody is equal before the law .  <eos>
번역: i ' m going to do my best .

한국어: 톰은 개를 쫓아갔어 . 
영어: <sos> tom ran after the dog .  <eos>
번역: i ' ve never seen anything to take you .

한국어: 조심해 ! 
영어: <sos> look out !  <eos>
번역: tom was diagnosed with asd .

한국어: 너 한때 여기서 일했었지 ? 
영어: <sos> you used to work here didn ' t you ?  <eos>
번역: what ' s the subway station ?

한국어: 톰은 알코올이라면 전혀 입에 대지 않아 . 
영어: <sos> tom doesn ' t drink alcoh